**CS560 - Algorithms and Their Analysis**
<br>
Date: **2 December 2020**
<br>

Title: **Lecture 12**
<br>
Speaker: **Dr. Shota Tsiskaridze**
<br>
Teaching Assistant: **Levan Sanadiradze**

Bibliography:
<br> 
 **Chapter 24**. Cormen, Thomas H. and Leiserson, Charles Eric and Rivest, Ronald Linn and Stein, Clifford Seth, *Introduction to Algorithms, 3rd Edition*, MIT Press, 2009
 


<h1 align="center">Single-Source Shortest Paths</h1>

<h3 align="center">Motivation</h3>

- **Professor Patrick** wishes to find the shortest possible route from **Phoenix** to **Indianapolis**.


- Given a **road map** of the **United States** on which the distance between each pair of adjacent intersections is marked.


- How can she **determine** this **shortest route**?

- we can model the **road map** as a **graph** $G = (V, E)$: 

  **vertices** represent **intersections**, 
  
  **edges** represent **road segments** between intersections, 
  
  **edge weights** represent road **distances**. 
  

-  Our **goal** is to **find a shortest path** from a given intersection in **Phoenix** to a given intersection in **Indianapolis**.

<h3 align="center">Problem Statement</h3>

- In a **shortest-paths problem**, we are given a **weighted**, **directed graph** $G =(V, E)$, **weight function** $w : E \rightarrow \mathbb{R}$ mapping edges to real-valued weights. 


- The **weight** $w(o)$ of path $p = \left \langle v_0, v_1, ..., v_k \right \rangle$ is the **sum of the weights** of its constituent **edges**:

  $$w(p) = \sum_{i=1}^{k} w(v_{i-1}, v_i).$$
  
- We **define** the **shortest-path weight** $\delta (u, v)$ from $u$ to $v$ by:

  $$\delta(u,v) = 
  \left\{\begin{matrix}
  \min \{w(p): u \leadsto^{p} v\} & \text{ if there is a path from} u \text{ to  } v ,\\
  \infty & \text{ otherwise.}
  \end{matrix}\right.$$
  
  
- A **shortest path** from vertex $u$ to vertex $v$ is then defined as **any path** $p$ with **weight** $w(p) = \delta (u,v)$.


- **Note**:

  Edge **weights** can represent **metrics** other than **distances**, such as **time**, **cost**, **penalties**, or **any other quantity** that accumulates linearly along a path.
  
   The **breadth-first-search algorithm** from is a **shortest-paths algorithm** that works on **unweighted graphs**.

<h3 align="center">Variants</h3>

- We shall focus on the **single-source shortest-paths problem**: 

  Given a graph $G = (V, E)$, we want to **find a shortest path** from a given **source vertex** $s \in V$ to each vertex $u \in V$. 
  
  
- The **algorithm** for the **single-source problem** can **solve many other problems**.


- **Single-destination shortest-paths problem**: 

  **Find a shortest path** to a given **destination** vertex $t$ from each vertex $v$. 
  
  By reversing the direction of each edge in the graph, we can reduce this problem to a single-source problem.
  
  
- **Single-pair shortest-path problem**: 

  **Find a shortest path** from $u$ to $v$ for given **vertices** $u$ and $v$.
  
  If we solve the single-source problem with source vertex $u$, we solve this problem also. 
  
  Moreover, all known algorithms for this problem have the same worst-case asymptotic running time as the best single-source algorithms.
  
  
- **All-pairs shortest-paths problem**: 

  **Find a shortest path** from $u$ to $v$ for every pair of vertices $u$ and $v$. 
  
  Although we can solve this problem by running a **singlesource algorithm** once from each vertex, we usually can solve it faster.

<h3 align="center">Optimal Substructure of a Shortest Path</h3>

- **Shortest-paths algorithms** typically **rely** on the **property** that a **shortest path** between two vertices **contains other shortest paths** within it.


- **Lemma (Subpaths of shortest paths are shortest paths)**:

  Given a **weighted**, **directed graph** $G = (V, E)$ with weight function $w :E \rightarrow \mathbb{R}$.
  
  Let $p = \left \langle v_0, v_1, ..., v_k \right \rangle$ be a shortest path from vertex $v_0$ to vertex $v_k$.
  
  For any $i$ and $j$ such that $0 \leq i \leq j \leq k$, let $p_{ij} = \left \langle v_i, v_{i+1}, ..., v_j \right \rangle$ be the subpath of $p$ from vertex $v_i$ to vertex $v_j$. 
  
  Then, $p_{ij}$ is a **shortest path** from $v_i$ to $v_j$.

<h3 align="center">Negative-Weight Edges</h3>

- **Some instances** of the single-source shortest-paths problem **may include edges** whose **weights are negative**. 


- If the graph $G = (V, E)$ contains **no negativeweight cycles** reachable from the **source** $s$, then for all $v \in V$, the shortest-path weight $\delta(s, v)$ remains well defined, even if it has a negative value. 


- If the graph contains a **negative-weight cycle** reachable from $s$, however, shortest-path weights are **not well defined**. 

  If there is a **negativeweight cycle** on some path from $s$ to $v$, we define $\delta (s, v) = - \infty$.
  
  <img src="images/L12_NG.png" width="800" alt="Example" />

  

<h3 align="center">Cycles</h3>

- Can a **shortest path** contain a **cycle**?


- As we have just seen, it **cannot contain** a **negative-weight cycle**. 


- **Nor can it contain** a **positive-weight cycle**, since removing the cycle from the path produces a path with the same source and destination vertices and a lower path weight. 

  That is, if $p = \left \langle v_0, v_1, ..., v_k \right \rangle$ is a path and $q = \left \langle v_i, v_{i+1}, ..., v_j \right \rangle$ is a positive-weight cycle on this path (so that $v_i = v_j$ and $w(q) > 0 $), then the path $r = \left \langle v_0, v_1, ..., v_i, v_{j+1}, ...,  v_k \right \rangle$ has weight $w(r) = w(p) - w(q)$, and so $p$ cannot be a shortest path from $v_0$ to $v_k$.
  
  
- That leaves only **0-weight cycles**, that we can remove from any path to produce another path whose weight is the same.


- Therefore, **without loss of generality** we can assume that when we are finding **shortest paths**, they have **no cycles**. 

  Since any acyclic path in a graph $G = (V, E)$ contains at most $|V|$ distinct vertices, it also contains at most $|V|-1j$ edges. 
  
  Thus, we can **restrict our attention** to **shortest paths of at most $|V| - 1$ edges**.

<h3 align="center">Relaxation</h3>

- For each vertex $v\in V$, we maintain an attribute $v.d$, which is an upper bound on the weight of a shortest path from source $s$ to $v$.


- We call $v.d$ a **shortest-path estimate**. 


- We **initialize** the shortest-path estimates and predecessors by the following $\Theta(V)$-time procedure:

  <img src="images/L12_Pseudocode_initialize.png" width="400" alt="Example" />
  
  After initialization, we have $v.\pi = NIL$ for all $v \in V$, $s.d = 0$ and $v.d = \infty$ for $v \in V - \{s\}$.

- The process of **relaxing** an edge $(u,v)$ consists of testing whether we can improve the shortest path to $v$ found so far by going through $u$.

  If so, updating $v.d$ and $v.\pi$.


- A **relaxation step** may decrease the value of the shortest-path estimate $v.d$ and update $v$’s predecessor attribute $v.\pi$. 


- The following code performs a relaxation step on edge $(u,v)$ in $O(1)$ time:

  <img src="images/L12_Pseudocode_Relax.png" width="400" alt="Example" />



<h3 align="center">The Bellman-Ford Algorithm</h3>

- The **Bellman-Ford algorithm** solves the **single-source shortest-paths problem** in the **general case** in which **edge weights may be negative**. 


- Given a **weighted**, **directed graph** $G = (V,E)$ with **source** $s$ and **weight function** $w: E \rightarrow \mathbb{R}$.


- The Bellman-Ford algorithm **returns a boolean value** indicating **whether or not** there is a **negative-weight cycle** that is reachable from the source. 

  If **there is** such a cycle, the **algorithm indicates that no solution exists**. 
  
  If **there is no** such cycle, the algorithm **produces the shortest paths** and their weights.
  
  
- The **algorithm relaxes edges**, progressively **decreasing an estimate** $v.d$ on the weight of a shortest path from the source $s$ to each vertex $v \in V$ **until** it **achieves the actual shortest-path weight** $\delta(s,v)$. 


- The algorithm returns $TRUE$ if and only if the **graph contains no negative-weight cycles** that are reachable from the source:

  <img src="images/L12_Pseudocode_Bellman_Ford.png" width="500" alt="Example" />

- The Bellman-Ford algorithm **runs in time** $O(VE)$, since:

   The initialization in **line 1** takes $\Theta(V)$ time;
  
   Each of the $|V| - 1$ passes over the edges in **lines 2–4** takes $\Theta(E)$ time;
  
   The **for** loop of **lines 5–7** takes $O(E)$ time.


- The execution of the Bellman-Ford algorithm is shown below:

  <img src="images/L12_Execution_Bellman_Ford.png" width="1000" alt="Example" />



<h3 align="center">Single-Source Shortest Paths in Directed Acyclic Graphs</h3>

- By relaxing the edges of a **weighted directed acyclic graph** (**dag**) $G = (V, E)$ according to a **topological sort** of its vertices, we can compute shortest paths from a single source in $\Theta(V + E)$ time. 


- **Shortest paths** are **always well defined** in a **dag**, since even if there are negative-weight edges, no negative-weight cycles can exist.


- The **algorithm starts** by **topologically sorting** the dag to impose a linear ordering on the vertices. 


- If the dag contains a path from vertex $u$ to vertex $v$, then $u$ precedes $v$ in the **topological sort**. We make just one pass over the vertices in the topologically sorted order. 


 - As we **process each vertex**, we **relax each edge** that leaves the vertex.

  <img src="images/L12_Pseudocode_DAG.png" width="600" alt="Example" />

- The running time of this algorithm is easy to analyze. 

  The **topological sort** of **line 1** takes $\Theta(V + E)$ time. 
  
  The call of **INITIALIZESINGLE SOURCE** in **line 2** takes $\Theta(V)$ time. 
  
  The **for** loop of **lines 3–5* makes one iteration per vertex. 
  
  Altogether, the for loop of **lines 4–5** relaxes each edge exactly once.
  
  Because each iteration of the inner **for** loop takes $\Theta(1)$ time, the **total running time** is $\Theta(V + E)$.


- The execution of the **DAG-Shortest-Path** algorithm is shown below:

  <img src="images/L12_Execution_DAG.png" width="1000" alt="Example" />



<h3 align="center">Dijkstra’s Algorithm</h3>

- **Dijkstra’s algorithm** solves the **single-source shortest-paths problem** on a **weighted**, **directed graph** $G = (V, E)$ for the case in which all **edge weights are nonnegative**.



- We assume that $w(u, v) \geq 0$ for each edge $(u,v) \in E$. 


- Dijkstra’s algorithm **maintains a set** $S$ of vertices whose **final shortest-path weights** from the **source** $s$ have already been **determined**. 


- The algorithm repeatedly selects the vertex $u \in V - S$ with the **minimum shortest-path estimate**, adds $u$ to $S$, and **relaxes all edges leaving** $u$. 


-  In the following implementation, we use a **min-priority queue** $Q$ of vertices, **keyed** by their $d$ **values**.

  <img src="images/L12_Pseudocode_Dijekstra.png" width="500" alt="Example" />

- How fast is Dijkstra’s algorithm? 

  It maintains the **min-priority queue** $Q$ by calling three priority-queue operations: 
  
   - `INSERT` (implicit in **line 3**)
   
   - `EXTRACT-MIN` (**line 5**);
   
   - `DECREASE-KEY` (implicit in **RELAX**, which is called in **line 8**). 
   
   The algorithm calls both `INSERT` and `EXTRACT-MIN` once per vertex. 
   
   Because each vertex $u \in V$ is **added** to set S exactly **once**, each edge in the adjacency list $Adj[u]$ is examined in the for loop of **lines 7–8** exactly once during the course of the algorithm.
   
   Since the total number of edges in all the adjacency lists is $|E|$, this **for** loop iterates a total of $|E|$ times, and thus the algorithm calls `DECREASE-KEY` at most $|E|$ times overall.

  The **running time** of **Dijkstra’s algorithm** depends on how we implement the min-priority queue:

  Consider the case in which we maintain the **min-priority queue** by taking advantage of the vertices being numbered $1$ to $|V|$. 
  
  We simply store $v.d$ in the $v$-th entry of an array. 
  
  Each `INSERT` and `DECREASE-KEY` operation takes $O(1)$ time, and each `EXTRACT-MIN` operation takes $O(V)$ time.
  
  Thus, a **total running time** is $O(V^2 + E) = O(V^2)$.


- The execution of the **DAG-Shortest-Path** algorithm is shown below:

  <img src="images/L12_Execution_Dijekstra.png" width="1000" alt="Example" />



<h1 align="center">End of Lecture</h1>